In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd

# .py file with functions: addnoise
import preparedata

print(tf.__version__)

2.15.0


In [5]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
print(y_train.shape)
y_train

(60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [8]:
y_train_copy = np.copy(y_train)
print(y_train_copy.shape)
y_train_copy

(60000,)


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [9]:
import preparedata

# add noise to the data
y_train_indices = preparedata.addnoise(y_train, 10)

In [10]:
y_train = y_train_indices[0]
# store indices of noisy data
indices = y_train_indices[1]

In [11]:
# Count the number of differing values (sanity check)
differing_values_count = np.sum(y_train_copy != y_train)

print(f"Number of differing values: {differing_values_count}")

Number of differing values: 6000


### Model VGG-7


In [12]:
# normalzing data using maximum grey-scale value
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

In [13]:
# one hot encoding
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [14]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [15]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            (5, 5),
            padding="same",
            activation="relu",
            input_shape=input_shape,
            name="conv1_1",
        ),
        tf.keras.layers.Conv2D(
            32, (5, 5), padding="same", activation="relu", name="conv1_2"
        ),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_1"
        ),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_2"
        ),
        tf.keras.layers.MaxPool2D(strides=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu", name="FC_1"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu", name="FC_2"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", name="FC_softmax"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.SGD(name="SGD"),
    loss="categorical_crossentropy",
    metrics=["acc"],
)

In [16]:
optimizer = tf.keras.optimizers.SGD(name="SGD")
cce = tf.keras.losses.CategoricalCrossentropy()

Function to compute average norm gradients per layer

In [17]:
def avg_grd_norms(variables, gradients):
    # take norm of each gradient, average them, and normalize with sqrt(no_gradients)
    norms = []
    names = []
    index = 0
    for var, grad in zip(model.trainable_variables, gradients):
        layer_size = grad.shape[-1]
        normalizer = np.sqrt(grad.shape[-1])
        # index 0,2,4,6 are conv layers
        if index in [0, 2, 4, 6]:
            l2 = []
            for i in range(layer_size):
                flattened_weights = tf.reshape(grad[:, :, :, i], [-1])
                normalized_norm = (
                    tf.norm(flattened_weights, ord="euclidean") / normalizer
                )
                l2.append(normalized_norm)

            norms.append(sum(l2))
            names.append(var.name)
            index += 1
        # index 8,10,12 are fully connected layer
        elif index in [8, 10, 12]:
            l2 = []
            for i in range(layer_size):
                flattened_weights = tf.reshape(grad[:, i], [-1])
                normalized_norm = (
                    tf.norm(flattened_weights, ord="euclidean") / normalizer
                )
                l2.append(normalized_norm)
            norms.append(normalized_norm)
            names.append(var.name)
            index += 1
        # remaining indices are bias weights
        else:
            index += 1

    return (names, norms)

### Training, logging gradients per epoch and layer

In [18]:
from keras.metrics import CategoricalAccuracy

accuracy_metric = CategoricalAccuracy()

epoch_gradients = []
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for batch_start in range(0, len(x_train), batch_size):
        # Extract a batch of data
        X_batch = x_train[batch_start : batch_start + batch_size]
        y_batch = y_train[batch_start : batch_start + batch_size]

        # Forward pass and compute gradients
        with tf.GradientTape() as tape:
            predictions = model(X_batch)
            loss = cce(y_batch, predictions)
            loss_value = tf.reduce_mean(loss)

        # Backward pass and update weights
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Get accuracy of the epoch
        accuracy_metric.update_state(y_batch, predictions)

    accuracy = accuracy_metric.result()
    accuracy_metric.reset_states()  # reset for next epoch

    epoch_gradients.append(gradients)

    print(f"  Loss: {loss_value.numpy()}, Acc: {accuracy.numpy()}")

Epoch 1/30


  Loss: 0.09363843500614166, Acc: 0.6846833229064941
Epoch 2/30
  Loss: 0.03076029382646084, Acc: 0.8603333234786987
Epoch 3/30
  Loss: 0.027629360556602478, Acc: 0.8748833537101746
Epoch 4/30
  Loss: 0.020832035690546036, Acc: 0.8821166753768921
Epoch 5/30
  Loss: 0.012489753775298595, Acc: 0.8875166773796082
Epoch 6/30
  Loss: 0.009639555588364601, Acc: 0.8891500234603882
Epoch 7/30
  Loss: 0.0065117208287119865, Acc: 0.8880500197410583
Epoch 8/30
  Loss: 0.004742208402603865, Acc: 0.8914333581924438
Epoch 9/30
  Loss: 0.006683826446533203, Acc: 0.888949990272522
Epoch 10/30
  Loss: 0.003002404235303402, Acc: 0.8894166946411133
Epoch 11/30
  Loss: 0.0031762367580085993, Acc: 0.89410001039505
Epoch 12/30
  Loss: 0.0030186506919562817, Acc: 0.8968333601951599
Epoch 13/30
  Loss: 0.001982476795092225, Acc: 0.8988833427429199
Epoch 14/30
  Loss: 0.0020076928194612265, Acc: 0.8998000025749207
Epoch 15/30
  Loss: 0.001372514059767127, Acc: 0.901116669178009
Epoch 16/30
  Loss: 0.0010483575

In [19]:
gradients_df = pd.DataFrame(columns=["name"])
for index in range(0, 30):
    column = f"epoch_{index}"
    gradients = epoch_gradients[index]
    names, norms = avg_grd_norms(model.trainable_variables, gradients)

    gradients_df[column] = norms

gradients_df["name"] = names

In [20]:
gradients_df["average"] = gradients_df.iloc[:, 1:].mean(axis=1)

In [21]:
gradients_df

,name,epoch_0,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,epoch_6,epoch_7,epoch_8,...,epoch_21,epoch_22,epoch_23,epoch_24,epoch_25,epoch_26,epoch_27,epoch_28,epoch_29,average
0,conv1_1/kernel:0,"tf.Tensor(0.16422333, shape=(), dtype=float32)","tf.Tensor(0.06501867, shape=(), dtype=float32)","tf.Tensor(0.066425495, shape=(), dtype=float32)","tf.Tensor(0.056950152, shape=(), dtype=float32)","tf.Tensor(0.038015284, shape=(), dtype=float32)","tf.Tensor(0.02758797, shape=(), dtype=float32)","tf.Tensor(0.020586178, shape=(), dtype=float32)","tf.Tensor(0.015022803, shape=(), dtype=float32)","tf.Tensor(0.020230392, shape=(), dtype=float32)",...,"tf.Tensor(0.013215568, shape=(), dtype=float32)","tf.Tensor(0.012281542, shape=(), dtype=float32)","tf.Tensor(0.009646066, shape=(), dtype=float32)","tf.Tensor(0.007202548, shape=(), dtype=float32)","tf.Tensor(0.0040155714, shape=(), dtype=float32)","tf.Tensor(0.0029645162, shape=(), dtype=float32)","tf.Tensor(0.0035697205, shape=(), dtype=float32)","tf.Tensor(0.0049784817, shape=(), dtype=float32)","tf.Tensor(0.0040355003, shape=(), dtype=float32)",0.020319
1,conv1_2/kernel:0,"tf.Tensor(0.25742742, shape=(), dtype=float32)","tf.Tensor(0.11967651, shape=(), dtype=float32)","tf.Tensor(0.13171764, shape=(), dtype=float32)","tf.Tensor(0.12085417, shape=(), dtype=float32)","tf.Tensor(0.080802895, shape=(), dtype=float32)","tf.Tensor(0.061946623, shape=(), dtype=float32)","tf.Tensor(0.041605137, shape=(), dtype=float32)","tf.Tensor(0.027498178, shape=(), dtype=float32)","tf.Tensor(0.048803892, shape=(), dtype=float32)",...,"tf.Tensor(0.030789578, shape=(), dtype=float32)","tf.Tensor(0.025624214, shape=(), dtype=float32)","tf.Tensor(0.018883375, shape=(), dtype=float32)","tf.Tensor(0.013603615, shape=(), dtype=float32)","tf.Tensor(0.0076745776, shape=(), dtype=float32)","tf.Tensor(0.005819016, shape=(), dtype=float32)","tf.Tensor(0.008919535, shape=(), dtype=float32)","tf.Tensor(0.011191608, shape=(), dtype=float32)","tf.Tensor(0.008933329, shape=(), dtype=float32)",0.039062
2,conv2_1/kernel:0,"tf.Tensor(0.38824803, shape=(), dtype=float32)","tf.Tensor(0.23866479, shape=(), dtype=float32)","tf.Tensor(0.3049998, shape=(), dtype=float32)","tf.Tensor(0.2580721, shape=(), dtype=float32)","tf.Tensor(0.16285004, shape=(), dtype=float32)","tf.Tensor(0.12006779, shape=(), dtype=float32)","tf.Tensor(0.07814045, shape=(), dtype=float32)","tf.Tensor(0.057965968, shape=(), dtype=float32)","tf.Tensor(0.10753118, shape=(), dtype=float32)",...,"tf.Tensor(0.062154066, shape=(), dtype=float32)","tf.Tensor(0.051419593, shape=(), dtype=float32)","tf.Tensor(0.04323868, shape=(), dtype=float32)","tf.Tensor(0.031906188, shape=(), dtype=float32)","tf.Tensor(0.017869122, shape=(), dtype=float32)","tf.Tensor(0.012335395, shape=(), dtype=float32)","tf.Tensor(0.021429935, shape=(), dtype=float32)","tf.Tensor(0.026702145, shape=(), dtype=float32)","tf.Tensor(0.021307448, shape=(), dtype=float32)",0.076672
3,conv2_2/kernel:0,"tf.Tensor(0.60732645, shape=(), dtype=float32)","tf.Tensor(0.41954425, shape=(), dtype=float32)","tf.Tensor(0.5012076, shape=(), dtype=float32)","tf.Tensor(0.43881765, shape=(), dtype=float32)","tf.Tensor(0.28254005, shape=(), dtype=float32)","tf.Tensor(0.21914051, shape=(), dtype=float32)","tf.Tensor(0.13634314, shape=(), dtype=float32)","tf.Tensor(0.100222334, shape=(), dtype=float32)","tf.Tensor(0.1740576, shape=(), dtype=float32)",...,"tf.Tensor(0.11069904, shape=(), dtype=float32)","tf.Tensor(0.09699357, shape=(), dtype=float32)","tf.Tensor(0.08189589, shape=(), dtype=float32)","tf.Tensor(0.059225287, shape=(), dtype=float32)","tf.Tensor(0.03280283, shape=(), dtype=float32)","tf.Tensor(0.022102114, shape=(), dtype=float32)","tf.Tensor(0.037864223, shape=(), dtype=float32)","tf.Tensor(0.048557512, shape=(), dtype=float32)","tf.Tensor(0.038453504, shape=(), dtype=float32)",0.130302
4,FC_1/kernel:0,"tf.Tensor(0.018603515, shape=(), dtype=float32)","tf.Tensor(0.012248592, shape=(), dtype=float32)","tf.Tensor(

Saving relevant data to avoid re-training.

In [22]:
# saving gradients for 30 epochs
csv_file_path = "/content/drive/MyDrive/noisy_gradients.csv"

# Save the DataFrame as a CSV file on Google Drive
gradients_df.to_csv(csv_file_path, index=False)

# Display a message indicating the successful save
print(f"DataFrame saved as CSV on Google Drive at: {csv_file_path}")

DataFrame saved as CSV on Google Drive at: /content/drive/MyDrive/noisy_gradients.csv


In [26]:
# save noisy data
csv_file_path = "/content/drive/MyDrive/noisy_y_train.csv"

# Save as a CSV file on Google Drive
np.savetxt(csv_file_path, y_train.numpy(), delimiter=",")

In [27]:
# save indices at which data is changed
path = "/content/drive/MyDrive/noisy_indices.csv"

np.savetxt(path, indices, delimiter=",")

convert tensors in list of list to numpy arrays before saving as csv

In [39]:
numpy_array_list = [
    [layer_grad.numpy() for layer_grad in epoch] for epoch in epoch_gradients
]

In [54]:
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.save("/content/drive/MyDrive/epoch_layer_gradients.ckpt")

'/content/drive/MyDrive/epoch_layer_gradients.ckpt-1'

In [ ]:
model.predict()